In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
pwd

u'/Users/yuhantang/AM207Project/correct/am207finalproject/data'

In [2]:
houses = ['A', 'B', 'C']
xtype_dict = {'r':'raw', 'c':'change'}

In [3]:
def read_data(house, xtype):
    assert house in houses
    assert xtype in xtype_dict
    
    act_df = pd.read_csv("house{}_act.csv".format(house))
    sensor_df = pd.read_csv("house{}_sensor.csv".format(house))
    X = np.load("X_{}_house{}.npy".format(xtype_dict[xtype], house))
    Y = np.load("Y_house{}.npy".format(house))
    return act_df, sensor_df, X, Y

In [4]:
#each row represents sensors for a specific action i
#each column is sensor j
def calc_miu(X, Y, all_sensor, all_act):
    assert len(X)==len(Y)
    
    num_act = len(all_act)
    num_sensor = len(all_sensor)
    miu = np.zeros([num_act, num_sensor])
    for act in all_act:
        sensor = X[Y==act]
        for row in sensor:
            miu[a2i[act]] += row
        total = np.sum(Y==act)
        if total != 0:
            miu[a2i[act]] = miu[a2i[act]]/total
#         else:
#             miu[a2i[act]] = [1e-10]*len(miu[i])
    #normalize
#     for i,row in enumerate(miu):
#         total = np.sum(row)
#         if total != 0:
#             miu[i] = miu[i]/total
#         else:
#             miu[i] = [1e-10]*len(miu[i])
        
    return miu

In [5]:
def max_posterior(miu, X, prior):
    num_time = len(X)
    num_act = len(miu)
    prob = np.zeros((num_time, num_act))
    for i in range(num_time):
        sensor = X[i]
        for j in range(num_act):
            act = miu[j]
            logp = 0
            for k,s in enumerate(sensor):
                if s == 1.:
                    logp += np.log(act[k])
                else:
                    logp += np.log(1-act[k])
            prob[i,j] = logp + np.log(prior[i , j])
    return prob

In [6]:
def MAP(matrix):
    est = np.zeros(len(matrix))
    for i,row in enumerate(matrix):
        est[i] = np.argmax(row)
    return est

In [7]:
def precision(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TI = np.sum(pred_label==y)
        if TI != 0:
            res += (float(TP)/TI)
    return float(res)/N

def recall(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TT = np.sum(Y==y)
        if TT != 0:
            res += float(TP)/TT
    return float(res)/N

def f_score(pred_label, Y):
    p = precision(pred_label, Y)
    r = recall(pred_label, Y)
    return 2*p*r/(p+r)

def accuracy(pred_label, Y):
    res = 0
    all_label = list(set(Y))
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        res += TP
    return float(res)/len(Y)

# Raw house A MAP

In [8]:
#read in data
house = 'A'
t = 'r'
act_df,sensor_df,X,Y = read_data(house, t)

#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

priorA = np.load('Prior_A.npy')

miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

miu = calc_miu(X,Y,all_sensor, all_act)
print priorA.shape
likes = max_posterior(miu,X,priorA)
result = MAP(likes) #note: result is index, not label yet

#convert index to label
res_label = np.array([i2a[e] for e in result])
for i,act in enumerate(all_act):
    print "{},act label: {}, Y count:{}, estimate count:{}".format(i,act, sum(Y==act), sum(res_label==act))

# sensors:  14
# states/acts:  17
# timeframes:  40006
(40006, 18)
0,act label: 1.0, Y count:19723, estimate count:19546
1,act label: 3.0, Y count:0, estimate count:847
2,act label: 4.0, Y count:220, estimate count:225
3,act label: 5.0, Y count:240, estimate count:1246
4,act label: 6.0, Y count:31, estimate count:595
5,act label: 10.0, Y count:11670, estimate count:10356
6,act label: 13.0, Y count:71, estimate count:66
7,act label: 15.0, Y count:292, estimate count:676
8,act label: 16.0, Y count:33, estimate count:197
9,act label: 17.0, Y count:21, estimate count:72
10,act label: 18.0, Y count:38, estimate count:36
11,act label: 19.0, Y count:25, estimate count:25
12,act label: 20.0, Y count:2, estimate count:6
13,act label: 22.0, Y count:10, estimate count:152
14,act label: 23.0, Y count:11, estimate count:250
15,act label: 25.0, Y count:454, estimate count:196
16,act label: 0, Y count:7165, estimate count:5515


# Evaluation Raw A

$$\text{Precision} = \frac{1}{N}\sum^N_i \frac{TP_i}{TI_i} $$
$$\text{Recall} = \frac{1}{N}\sum^N_i\frac{TP_i}{TT_i} $$
$$\text{F-Measure} = \frac{2 \cdot precision \cdot recal}{precision+recall} $$
$$\text{Accuracy} = \frac{\sum^N_i TP_i}{Total}$$

In [9]:
print 'Precision of house {} is {}'.format(house,precision(res_label, Y))
print 'recall of house {} is {}'.format(house,recall(res_label, Y))
print 'F score of house {} is {}'.format(house,f_score(res_label, Y))
print 'Accuracy of house {} is {}'.format(house,accuracy(res_label, Y))

Precision of house A is 0.309029971498
recall of house A is 0.441306017007
F score of house A is 0.36350858268
Accuracy of house A is 0.652252162176


# Raw house B MAP

In [10]:
#read in data
house = 'B'
t = 'r'
act_df,sensor_df,X,Y = read_data(house, t)

#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

priorB = np.load('Prior_B.npy')

miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

miu = calc_miu(X,Y,all_sensor, all_act)
print priorB.shape
likes = max_posterior(miu,X,priorB)
result = MAP(likes) #note: result is index, not label yet

#convert index to label
res_label = np.array([i2a[e] for e in result])
for i,act in enumerate(all_act):
    print "{},act label: {}, Y count:{}, estimate count:{}".format(i,act, sum(Y==act), sum(res_label==act))

# sensors:  22
# states/acts:  25
# timeframes:  38900
(38900, 25)
0,act label: 1.0, Y count:12242, estimate count:3943
1,act label: 4.0, Y count:49, estimate count:475
2,act label: 5.0, Y count:115, estimate count:442
3,act label: 6.0, Y count:30, estimate count:421
4,act label: 9.0, Y count:9, estimate count:15
5,act label: 10.0, Y count:6057, estimate count:3863
6,act label: 11.0, Y count:28, estimate count:420
7,act label: 13.0, Y count:85, estimate count:322
8,act label: 15.0, Y count:90, estimate count:71
9,act label: 17.0, Y count:6, estimate count:400
10,act label: 24.0, Y count:27, estimate count:466
11,act label: 29.0, Y count:30, estimate count:1612
12,act label: 31.0, Y count:50, estimate count:765
13,act label: 32.0, Y count:137, estimate count:1146
14,act label: 33.0, Y count:35, estimate count:140
15,act label: 34.0, Y count:38, estimate count:470
16,act label: 35.0, Y count:77, estimate count:513
17,act label: 36.0, Y count:4, estimate count:19
18,act label: 37.0, Y cou

# Evaluation Raw B

In [11]:
print 'Precision of house {} is {}'.format(house,precision(res_label, Y))
print 'recall of house {} is {}'.format(house,recall(res_label, Y))
print 'F score of house {} is {}'.format(house,f_score(res_label, Y))
print 'Accuracy of house {} is {}'.format(house,accuracy(res_label, Y))

Precision of house B is 0.225435391479
recall of house B is 0.569163569009
F score of house B is 0.322954391776
Accuracy of house B is 0.656632390746


# Raw House C MAP

In [12]:
#read in data
house = 'C'
t = 'r'
act_df,sensor_df,X,Y = read_data(house, t)

#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

priorC = np.load('Prior_C.npy')

miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

miu = calc_miu(X,Y,all_sensor, all_act)
print priorC.shape
likes = max_posterior(miu,X,priorC)
result = MAP(likes) #note: result is index, not label yet

#convert index to label
res_label = np.array([i2a[e] for e in result])
for i,act in enumerate(all_act):
    print "{},act label: {}, Y count:{}, estimate count:{}".format(i,act, sum(Y==act), sum(res_label==act))

# sensors:  21
# states/acts:  18
# timeframes:  26488
(26488, 18)
0,act label: 1.0, Y count:11961, estimate count:11622
1,act label: 3.0, Y count:352, estimate count:189
2,act label: 4.0, Y count:78, estimate count:257
3,act label: 5.0, Y count:192, estimate count:508
4,act label: 6.0, Y count:92, estimate count:395
5,act label: 7.0, Y count:56, estimate count:173
6,act label: 9.0, Y count:62, estimate count:379
7,act label: 10.0, Y count:7760, estimate count:6625
8,act label: 11.0, Y count:88, estimate count:267
9,act label: 12.0, Y count:9, estimate count:886
10,act label: 13.0, Y count:83, estimate count:353
11,act label: 14.0, Y count:65, estimate count:349
12,act label: 15.0, Y count:306, estimate count:139
13,act label: 16.0, Y count:8, estimate count:537
14,act label: 17.0, Y count:22, estimate count:349
15,act label: 22.0, Y count:40, estimate count:83
16,act label: 28.0, Y count:2457, estimate count:1457
17,act label: 0, Y count:2857, estimate count:1920


In [13]:
print 'Precision of house {} is {}'.format(house,precision(res_label, Y))
print 'recall of house {} is {}'.format(house,recall(res_label, Y))
print 'F score of house {} is {}'.format(house,f_score(res_label, Y))
print 'Accuracy of house {} is {}'.format(house,accuracy(res_label, Y))

Precision of house C is 0.165571904768
recall of house C is 0.27077995426
F score of house C is 0.20549266319
Accuracy of house C is 0.528239202658


# Change House A MAP

In [14]:
#read in data
house = 'A'
t = 'c'
act_df,sensor_df,X,Y = read_data(house, t)

#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

priorA = np.load('Prior_A.npy')

miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

miu = calc_miu(X,Y,all_sensor, all_act)
print priorA.shape
likes = max_posterior(miu,X,priorA)
result = MAP(likes) #note: result is index, not label yet

#convert index to label
res_label = np.array([i2a[e] for e in result])
for i,act in enumerate(all_act):
    print "{},act label: {}, Y count:{}, estimate count:{}".format(i,act, sum(Y==act), sum(res_label==act))

# sensors:  14
# states/acts:  17
# timeframes:  40006
(40006, 18)
0,act label: 1.0, Y count:19723, estimate count:22764
1,act label: 3.0, Y count:0, estimate count:495
2,act label: 4.0, Y count:220, estimate count:229
3,act label: 5.0, Y count:240, estimate count:1224
4,act label: 6.0, Y count:31, estimate count:183
5,act label: 10.0, Y count:11670, estimate count:13095
6,act label: 13.0, Y count:71, estimate count:38
7,act label: 15.0, Y count:292, estimate count:28
8,act label: 16.0, Y count:33, estimate count:124
9,act label: 17.0, Y count:21, estimate count:100
10,act label: 18.0, Y count:38, estimate count:18
11,act label: 19.0, Y count:25, estimate count:54
12,act label: 20.0, Y count:2, estimate count:30
13,act label: 22.0, Y count:10, estimate count:6
14,act label: 23.0, Y count:11, estimate count:8
15,act label: 25.0, Y count:454, estimate count:24
16,act label: 0, Y count:7165, estimate count:1586


In [15]:
print 'Precision of house {} is {}'.format(house,precision(res_label, Y))
print 'recall of house {} is {}'.format(house,recall(res_label, Y))
print 'F score of house {} is {}'.format(house,f_score(res_label, Y))
print 'Accuracy of house {} is {}'.format(house,accuracy(res_label, Y))

Precision of house A is 0.325030981642
recall of house A is 0.404310656557
F score of house A is 0.360361955787
Accuracy of house A is 0.596860470929


# Change House B MAP

In [16]:
#read in data
house = 'B'
t = 'c'
act_df,sensor_df,X,Y = read_data(house, t)

#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

priorB = np.load('Prior_B.npy')

miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

miu = calc_miu(X,Y,all_sensor, all_act)
print priorB.shape
likes = max_posterior(miu,X,priorB)
result = MAP(likes) #note: result is index, not label yet

#convert index to label
res_label = np.array([i2a[e] for e in result])
for i,act in enumerate(all_act):
    print "{},act label: {}, Y count:{}, estimate count:{}".format(i,act, sum(Y==act), sum(res_label==act))

# sensors:  22
# states/acts:  25
# timeframes:  38900
(38900, 25)
0,act label: 1.0, Y count:12242, estimate count:16275
1,act label: 4.0, Y count:49, estimate count:175
2,act label: 5.0, Y count:115, estimate count:23
3,act label: 6.0, Y count:30, estimate count:44
4,act label: 9.0, Y count:9, estimate count:32
5,act label: 10.0, Y count:6057, estimate count:545
6,act label: 11.0, Y count:28, estimate count:71
7,act label: 13.0, Y count:85, estimate count:34
8,act label: 15.0, Y count:90, estimate count:42
9,act label: 17.0, Y count:6, estimate count:49
10,act label: 24.0, Y count:27, estimate count:50
11,act label: 29.0, Y count:30, estimate count:332
12,act label: 31.0, Y count:50, estimate count:220
13,act label: 32.0, Y count:137, estimate count:104
14,act label: 33.0, Y count:35, estimate count:21
15,act label: 34.0, Y count:38, estimate count:39
16,act label: 35.0, Y count:77, estimate count:87
17,act label: 36.0, Y count:4, estimate count:63
18,act label: 37.0, Y count:14, esti

In [17]:
print 'Precision of house {} is {}'.format(house,precision(res_label, Y))
print 'recall of house {} is {}'.format(house,recall(res_label, Y))
print 'F score of house {} is {}'.format(house,f_score(res_label, Y))
print 'Accuracy of house {} is {}'.format(house,accuracy(res_label, Y))

Precision of house B is 0.197621602196
recall of house B is 0.232950889076
F score of house B is 0.213836828248
Accuracy of house B is 0.425732647815


# Change House C MAP

In [18]:
#read in data
house = 'C'
t = 'c'
act_df,sensor_df,X,Y = read_data(house, t)

#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

priorC = np.load('Prior_C.npy')

miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

miu = calc_miu(X,Y,all_sensor, all_act)
print priorC.shape
likes = max_posterior(miu,X,priorC)
result = MAP(likes) #note: result is index, not label yet

#convert index to label
res_label = np.array([i2a[e] for e in result])
for i,act in enumerate(all_act):
    print "{},act label: {}, Y count:{}, estimate count:{}".format(i,act, sum(Y==act), sum(res_label==act))

# sensors:  21
# states/acts:  18
# timeframes:  26488
(26488, 18)
0,act label: 1.0, Y count:11961, estimate count:14072
1,act label: 3.0, Y count:352, estimate count:103
2,act label: 4.0, Y count:78, estimate count:206
3,act label: 5.0, Y count:192, estimate count:898
4,act label: 6.0, Y count:92, estimate count:25
5,act label: 7.0, Y count:56, estimate count:124
6,act label: 9.0, Y count:62, estimate count:39
7,act label: 10.0, Y count:7760, estimate count:8979
8,act label: 11.0, Y count:88, estimate count:95
9,act label: 12.0, Y count:9, estimate count:156
10,act label: 13.0, Y count:83, estimate count:58
11,act label: 14.0, Y count:65, estimate count:33
12,act label: 15.0, Y count:306, estimate count:106
13,act label: 16.0, Y count:8, estimate count:73
14,act label: 17.0, Y count:22, estimate count:75
15,act label: 22.0, Y count:40, estimate count:34
16,act label: 28.0, Y count:2457, estimate count:287
17,act label: 0, Y count:2857, estimate count:1125


In [19]:
print 'Precision of house {} is {}'.format(house,precision(res_label, Y))
print 'recall of house {} is {}'.format(house,recall(res_label, Y))
print 'F score of house {} is {}'.format(house,f_score(res_label, Y))
print 'Accuracy of house {} is {}'.format(house,accuracy(res_label, Y))

Precision of house C is 0.202923803297
recall of house C is 0.230502704228
F score of house C is 0.215835831912
Accuracy of house C is 0.590003020236
